In [1]:
from datetime import datetime, date, timedelta
import numpy as np
import xarray as xr
import sys
sys.path.append('/home/ascherrmann/scripts/')
import helper
import argparse
import pickle
import os

---
## Create File with cyclone information of case study 17 

---
### Structure of d:
-  0 col:    Time
-  1 col:    ID
-  2 col:    LON
-  3 col:    LAT
-  4 col:    Distance
-  5 col:    Area
-  6 col:    Inpres
-  7 col:    Outpres
-  8 col:    baroclin
-  9 col:    meanth
- 10 col:    ctype
- 11 col:    bergeron
- 12 col:    bmaxdiff
---

In [3]:

datastruct = dict()

labels = np.array([])
Gdates = np.array([])
data = dict()

LON = np.round(np.linspace(-180,180,901),1)
LAT = np.round(np.linspace(0,90,226),1)

ft = date.toordinal(date(1950,1,1))

add = 'IFS17-CYC-case-study'

datastruct = dict()

labels = np.array([])
Gdates = np.array([])
data = dict()






## LOAD DATA
################################# MODIFIED BY MAX ############################################################################
# - Load my Data and my validation IDs 
# path_trackCYC = f'/net/helium/atmosdyn/IFS-1Y/{Month}/features/tracking/TRACKED_CYCLONES'
path_trackCYC = f'/net/helium/atmosdyn/freimax/data_msc/IFS-17/trajectories/ctraj/TRACKED_CYCLONE.txt'
d = np.loadtxt(path_trackCYC,skiprows=1)
IDs = np.unique(d[1])
print(f'IDs from the TRACKED_CYCLONE file:\n{IDs}')

IDs from the TRACKED_CYCLONE file:
[0.]


In [5]:


# ## Load validation cyclone IDs
# path_CYCsplit_info = f'/net/helium/atmosdyn/freimax/data_msc/IFS-18/cyclones/data_random_forest/{MOT}/cyclone_split_info.txt'
# with open(path_CYCsplit_info, 'r') as f:
#     lines = f.readlines()
# num_cyclones = int(lines[0].split(': ')[1])  # The 'Number of cyclones' line
# id_list_str = lines[1].split(': ')[1].strip('[]\n')
# id_list_cyc = [float(x) for x in id_list_str.split(', ')] if id_list_str else []  # The 'ID list' line
# id_val_str = lines[2].split(': ')[1].strip('[]\n')
# IDs_valCYC = [float(x) for x in id_val_str.split(', ')] if id_val_str else []  # The 'ID of cyclones in validation set' line
# print(f'IDs of the validation cyclones:\n{MOT}\t{IDs_valCYC}\n')
# ################################################################################################################################


In [7]:
ids = IDs[0]

## cappear contains all indices of rows in d that have ids (one of the validation IDs)        
cappear, = np.where((d[1]==ids))


clat = []
clon = []
hourszeta = np.array([])
hourstoSLPmin = np.array([])
zetal = np.array([])
dates = np.array([])
zeta = np.zeros(len(cappear))
hours = np.zeros(len(cappear))


hoursslp = np.array([])         # MAX
slpl = np.array([])             # MAX
slp = np.zeros(len(cappear))    # MAX

# find id of minimum SLP in track file
# slpminid = cappear[np.where(d[cappear,6] == np.min(d[cappear,6]))]
# slpminid = slpminid[-1]
# hourstoSLPmin = np.append(hourstoSLPmin, d[cappear,0]-d[slpminid,0])


In [8]:
### step through hrs of the cyclon in trackfile
# for wr,u in enumerate(cappear):
wr, u = 0, 0
tmp = d
Date = '20170410_17'

# Load dataset with PS, SLP and Vorticity
dataset_1 = xr.open_dataset(f'/net/helium/atmosdyn/freimax/data_msc/IFS-17/cdf/P{Date}')
data[Date] = dataset_1

dates = np.append(dates,Date)
Gdates = np.append(Gdates,Date)

In [12]:
### keep adding lon to lon and lat to lat and use srtid, as it is correct with normal crosssections
CLONIDS, CLATIDS = helper.IFS_radial_ids_correct(200,LAT[np.where(LAT==np.round(tmp[3],1))[0][0]])
addlon = CLONIDS + np.where(LON==np.round(tmp[2],1))[0][0]
addlon[np.where((addlon-900)>0)] = addlon[np.where((addlon-900)>0)]-900

clat.append(CLATIDS.astype(int) + np.where(LAT==np.round(tmp[3],1))[0][0])
clon.append(addlon.astype(int))

In [24]:
    slpl = np.append(slpl, data[Date].SLP.isel(lat=slice(clat[-1][0]+225,clat[-1][-1]+225) , lon=slice(clon[-1][0],clon[-1][-1])).values)
    
    slp[wr] = np.min(slpl)

    min_slp_t = data[Date].SLP.isel(lat=slice(clat[-1][0]+225,clat[-1][-1]+225) , lon=slice(clon[-1][0],clon[-1][-1])).values.min()
    print(f'{str(data[Date].time.values)[0:18]}\t\tMin slp: {min_slp_t:.1f}\tLonbox: [{data[Date].lon[clon[-1].min()].values:.1f}, {data[Date].lon[clon[-1].max()].values:.1f}]  |  Latbox: [{data[Date].lat[clat[-1].min()+225].values:.1f}, {data[Date].lat[(clat[-1]+225).max()].values:.1f}]')



2017-04-10T17:00:0		Min slp: 965.0	Lonbox: [160.8, 165.6]  |  Latbox: [45.2, 48.4]


In [25]:

### add 1 as clat, clon star            print(f'') with zeros as base to stack them
MatureLat = np.round(np.mean(LAT[clat[np.where(slp==np.min(slp))[0][-1]]]),1)
MatureLon = np.round(np.mean(LON[clon[np.where(slp==np.min(slp))[0][-1]]]),1)
print(MatureLat, MatureLon)

46.9 163.2


In [13]:
# #hours since first identification
# hours[wr] = tmp[0]


### use last lon and lat entries [-1] of center
PS = data[Date].PS.values[clat[-1],clon[-1]]
for e in range(len(CLATIDS)):
    P = helper.modellevel_to_pressure(PS[e])
    I = np.where(abs(P-850)==np.min(abs(P-850)))[0]
    I = I[0].astype(int)
    # zetal = np.append(zetal,data[Date].VORT.values[0,I,clat[-1][e],clon[-1][e]])

# zeta[wr] = np.mean(zetal)



# hours = hours - hours[np.where(zeta==np.max(zeta))]
# hourszeta = np.append(hourszeta,hours)

# # allow for at least 48h backward trajectories
# if (((int(dates[np.where(zeta==np.max(zeta))[0][-1]][6:8])>=3)) | (int(dates[np.where(zeta==np.max(zeta))[0][-1]][4:6])!=MONTHN[np.where(MONTHS==MOT)[0][0]])):
#     MatureLat = np.round(np.mean(LAT[clat[np.where(zeta==np.max(zeta))[0][-1]]]),1)
#     MatureLon = np.round(np.mean(LON[clon[np.where(zeta==np.max(zeta))[0][-1]]]),1)
    # use latest maximum relative vorticity
#           #   MatureLat = np.round(np.mean(LAT[clat[np.where(zeta==np.max(zeta))[0][-1]+1]]),1)
#           #   MatureLon = np.round(np.mean(LON[clon[np.where(zeta==np.max(zeta))[0][-1]+1]]),1)


In [19]:
d

array([1.49184360e+09, 0.00000000e+00, 1.63600006e+02, 4.72000010e+01,
       9.64964355e+02])

In [26]:


if((hours[0]<=(-0)) & (hours[-1]>=0)):          # MAX
    datastruct['clat'] = clat
    datastruct['clon'] = clon
    datastruct['zeta'] = zeta
    datastruct['hzeta'] = hourszeta
    datastruct['SLP'] = d[4]
    datastruct['hSLP'] = hourstoSLPmin
    datastruct['dates'] = dates
    datastruct['Matlat'] = MatureLat
    datastruct['Matlon'] = MatureLon
   
    datastruct['label'] = ids
    print('Saved cyclone ID:', ids)

else: 
    print(f'File not saved, becuae time to mature stage is not long enough: {hours[0]}, {hours[-1]} ')



Saved cyclone ID: 0.0


In [27]:
datastruct

{'clat': [array([113, 113, 113, 113, 113, 114, 114, 114, 114, 114, 114, 114, 114,
         114, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 115, 116,
         116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 116, 117,
         117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 118,
         118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 118, 119,
         119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 119, 120,
         120, 120, 120, 120, 120, 120, 120, 120, 120, 120, 121, 121, 121,
         121, 121, 121, 121])],
 'clon': [array([856, 857, 858, 859, 860, 854, 855, 856, 857, 858, 859, 860, 861,
         862, 853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 852,
         853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 852,
         853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 852,
         853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 864, 852,
         853, 854, 855, 856, 857, 858, 859, 860, 861, 862, 863, 

In [28]:
PATH = '/net/helium/atmosdyn/freimax/data_msc/IFS-17'

# Create the directory if it does not exist
os.makedirs(PATH, exist_ok=True)

f = open(PATH + '/' + add + '.txt','wb')
pickle.dump(datastruct,f)
f.close()

In [30]:
add

'IFS17-CYC-case-study'